# Pumped Hydro-electric Energy Storage

A Pumped Hydro-electric Energy Storage station (PHES) is constituted of two dams, an upper dam (UD) and a lower one (LD). When UD turbines water to LD it produces electical power. When UD pumps water from LD it consumes electrical power. The idea of such a facility is to store energy when electricity is cheap to release it when electricity is expensive. We just described here a rule based heuristic to manage a PHES. The goal of this notebook is to show how to use the Stochastic Dynamic Programming (SDP) and the Stochastic Dual Dynamic Programming (SDDP) algorithm to find how to manage optimally these two storages. The results are compared with the exact solution provided by the extensive formulation that solves the problem on a scenario tree.


<img src=https://upload.wikimedia.org/wikipedia/commons/9/9a/Pumpstor_racoon_mtn.jpg>
<center>source : https://en.wikipedia.org/wiki/Pumped-storage_hydroelectricity</center>

## Mathematical formulation:

Assume that we have a water inflow $w^t$ which arrives in UD between $t$ and $t+1$. This dam pumps a quantity $u_t$ of water when $u_t$ is positive and turbines a quantity $u_t$ of water when u_t is negative. A certain amount of water can be spilt when both dams are full and water inflow positive in UD. $s_t^1$ designates the water spillage in UD. We denote by $x_t^{1}$ and $x_t^{2}$ the state of charge of each storage at time $t$.

Their dynamic are therefor: 

$$ x_{t+1}^1 = x_{t}^1 + w^t + u_t - s_t^1 $$

$$ x_{t+1}^2 = x_{t}^2 - u_t $$

Thus, we could define the state:
$$ X_t = (x_1^t, x_2^t)$$

and the control:
$$U_t = (u_t, s_1) $$

The turbines produce a quantity of electricity proportionnal to the flow turbined, and this electricity is sold into the market at a price $c_t$. We suppose that the buying and the selling costs are equal at each time step. So we pay or gain at each timestep:

$$ C_t(X_t, U_t) = c_t  u_t $$

We want to maximize our expected gains, so we minimize the following quantity:

$$ J = \mathbb{E} \left[ \sum_{i=1}^{T_f} C_t(X_t, U_t) \right]$$ 

## Problem formulation: 

First, we need to import some modules: 

In [1]:
using JuMP, Clp, StochDynamicProgramming, PyCall
pygui(:qt)
using Plots

JuMP is the julia module for Mathematical Programming, 
  Clp the module calling a linear solver (can be replaced by CPLEX or Gurobi), 
 StochDynamicProgramming is a Julia module containing SDP and SDDP algorithms, 
 Gadfly is used here for plotting the results 
 

---
### Constants definition

Then, we define the constants of this problem: 

In [2]:
# Number of timesteps (as we manage the dams over a year, it is equal to the number of weeks):
TF = 3

# Capacity of dams: 
VOLUME_MAX = 30
VOLUME_MIN = 0

# Specify the maximum flow of turbines: 
CONTROL_MAX = 20
CONTROL_MIN = -20

# Some statistics about aleas (water inflow):
W_MAX = 15
W_MIN = 0
DW = 1

T0 = 1

# Define aleas' space:
N_ALEAS = Int(round(Int, (W_MAX - W_MIN) / DW + 1))
ALEAS = linspace(W_MIN, W_MAX, N_ALEAS);


Now, we generate a random process to simulate the evolution of electricity prices over a year: 

In [3]:
COST = 66*2.7*(1 + .5*(rand(TF-1) - .5));


We could plot the evolution of prices using matplotlib: 

In [4]:
plot(1:TF-1, COST)

[Plots.jl] Initializing backend: pyplot


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Created with matplotlib (http://matplotlib.org/) -->
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 1.0 -->
 
 
 <path d="M 31.78125 66.40625 
Q 24.171875 66.40625 20.328125 58.90625 
Q 16.5 51.421875 16.5 36.375 
Q 16.5 21.390625 20.328125 13.890625 
Q 24.171875 6.390625 31.78125 6.390625 
Q 39.453125 6.390625 43.28125 13.890625 
Q 47.125 21.390625 47.125 36.375 
Q 47.125 51.421875 43.28125 58.90625 
Q 39.453125 66.40625 31.78125 66.40625 
M 31.78125 74.21875 
Q 44.046875 74.21875 50.515625 64.515625 
Q 56.984375 54.828125 56.984375 36.375 
Q 56.984375 17.96875 50.515625 8.265625 
Q 44.046875 -1.421875 31.78125 -1.421875 
Q 19.53125 -1.421875 13.0625 8.265625 
Q 6.59375 17.96875 6.59375 36.375 
Q 6.59375 54.828125 13.0625 64.515625 
Q 19.53125 74.21875 31.78125 74.21875 
" id="DejaVuSans-30"/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 1.2 -->
 
 <path d="M 19.1875 8.296875 
L 53.609375 8.296875 
L 53.609375 0 
L 7.328125 0 
L 7.328125 8.296875 
Q 12.9375 14.109375 22.625 23.890625 
Q 32.328125 33.6875 34.8125 36.53125 
Q 39.546875 41.84375 41.421875 45.53125 
Q 43.3125 49.21875 43.3125 52.78125 
Q 43.3125 58.59375 39.234375 62.25 
Q 35.15625 65.921875 28.609375 65.921875 
Q 23.96875 65.921875 18.8125 64.3125 
Q 13.671875 62.703125 7.8125 59.421875 
L 7.8125 69.390625 
Q 13.765625 71.78125 18.9375 73 
Q 24.125 74.21875 28.421875 74.21875 
Q 39.75 74.21875 46.484375 68.546875 
Q 53.21875 62.890625 53.21875 53.421875 
Q 53.21875 48.921875 51.53125 44.890625 
Q 49.859375 40.875 45.40625 35.40625 
Q 44.1875 33.984375 37.640625 27.21875 
Q 31.109375 20.453125 19.1875 8.296875 
" id="DejaVuSans-32"/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 1.4 -->
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 1.6 -->
 
 <path d="M 33.015625 40.375 
Q 26.375 40.375 22.484375 35.828125 
Q 18.609375 31.296875 18.609375 23.390625 
Q 18.609375 15.53125 22.484375 10.953125 
Q 26.375 6.390625 33.015625 6.390625 
Q 39.65625 6.390625 43.53125 10.953125 
Q 47.40625 15.53125 47.40625 23.390625 
Q 47.40625 31.296875 43.53125 35.828125 
Q 39.65625 40.375 33.015625 40.375 
M 52.59375 71.296875 
L 52.59375 62.3125 
Q 48.875 64.0625 45.09375 64.984375 
Q 41.3125 65.921875 37.59375 65.921875 
Q 27.828125 65.921875 22.671875 59.328125 
Q 17.53125 52.734375 16.796875 39.40625 
Q 19.671875 43.65625 24.015625 45.921875 
Q 28.375 48.1875 33.59375 48.1875 
Q 44.578125 48.1875 50.953125 41.515625 
Q 57.328125 34.859375 57.328125 23.390625 
Q 57.328125 12.15625 50.6875 5.359375 
Q 44.046875 -1.421875 33.015625 -1.421875 
Q 20.359375 -1.421875 13.671875 8.265625 
Q 6.984375 17.96875 6.984375 36.375 
Q 6.984375 53.65625 15.1875 63.9375 
Q 23.390625 74.21875 37.203125 74.21875 
Q 40.921875 74.21875 44.703125 73.484375 
Q 48.484375 72.75 52.59375 71.296875 
" id="DejaVuSans-36"/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 1.8 -->
 
 <path d="M 31.78125 34.625 
Q 24.75 34.625 20.71875 30.859375 
Q 16.703125 27.09375 16.703125 20.515625 
Q 16.703125 13.921875 20.71875 10.15625 
Q 24.75 6.390625 31.78125 6.390625 
Q 38.8125 6.390625 42.859375 10.171875 
Q 46.921875 13.96875 46.921875 20.515625 
Q 46.921875 27.09375 42.890625 30.859375 
Q 38.875 34.625 31.78125 34.625 
M 21.921875 38.8125 
Q 15.578125 40.375 12.03125 44.71875 
Q 8.5 49.078125 8.5 55.328125 
Q 8.5 64.0625 14.71875 69.140625 
Q 20.953125 74.21875 31.78125 74.21875 
Q 42.671875 74.21875 48.875 69.140625 
Q 55.078125 64.0625 55.078125 55.328125 
Q 55.078125 49.078125 51.53125 44.71875 
Q 48 40.375 41.703125 38.8125 
Q 48.828125 37.15625 52.796875 32.3125 
Q 56.78125 27.484375 56.78125 20.515625 
Q 56.78125 9.90625 50.3125 4.234375 
Q 43.84375 -1.421875 31.78125 -1.421875 
Q 19.734375 -1.421875 13.25 4.234375 
Q 6.78125 9.90625 6.78125 20.515625 
Q 6.78125 27.484375 10.78125 32.3125 
Q 14.79687

---
### Dynamic, costs and aleas


---

We could now define the dynamic of our system: 

In [5]:
# Define dynamic of the dam:
function dynamic(t, x, u, w)
    return [x[1] + u[1] + w[1] - u[2], x[2] - u[1]]
end

dynamic (generic function with 1 method)

and the cost at time $t$:

In [6]:
# Define cost corresponding to each timestep:
function cost_t(t, x, u, w)
    return COST[t] * (u[1])
end

cost_t (generic function with 1 method)

Now, we build a function to simulate the evolution of water inflow $w_t$.




In [7]:
"""Build admissible scenarios for water inflow over the time horizon."""
function build_scenarios(n_scenarios::Int64, n_stages)
    scenarios = zeros(n_scenarios, n_stages)

    for scen in 1:n_scenarios
        scenarios[scen, :] = (W_MAX-W_MIN)*rand(n_stages)+W_MIN
    end
    return scenarios
end


build_scenarios (generic function with 1 method)

We could test our generator with one scenario: 

In [8]:
scenario = build_scenarios(1, TF-1);

plot(1:TF-1,scenario')

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Created with matplotlib (http://matplotlib.org/) -->
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 1.0 -->
 
 
 <path d="M 31.78125 66.40625 
Q 24.171875 66.40625 20.328125 58.90625 
Q 16.5 51.421875 16.5 36.375 
Q 16.5 21.390625 20.328125 13.890625 
Q 24.171875 6.390625 31.78125 6.390625 
Q 39.453125 6.390625 43.28125 13.890625 
Q 47.125 21.390625 47.125 36.375 
Q 47.125 51.421875 43.28125 58.90625 
Q 39.453125 66.40625 31.78125 66.40625 
M 31.78125 74.21875 
Q 44.046875 74.21875 50.515625 64.515625 
Q 56.984375 54.828125 56.984375 36.375 
Q 56.984375 17.96875 50.515625 8.265625 
Q 44.046875 -1.421875 31.78125 -1.421875 
Q 19.53125 -1.421875 13.0625 8.265625 
Q 6.59375 17.96875 6.59375 36.375 
Q 6.59375 54.828125 13.0625 64.515625 
Q 19.53125 74.21875 31.78125 74.21875 
" id="DejaVuSans-30"/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 1.2 -->
 
 <path d="M 19.1875 8.296875 
L 53.609375 8.296875 
L 53.609375 0 
L 7.328125 0 
L 7.328125 8.296875 
Q 12.9375 14.109375 22.625 23.890625 
Q 32.328125 33.6875 34.8125 36.53125 
Q 39.546875 41.84375 41.421875 45.53125 
Q 43.3125 49.21875 43.3125 52.78125 
Q 43.3125 58.59375 39.234375 62.25 
Q 35.15625 65.921875 28.609375 65.921875 
Q 23.96875 65.921875 18.8125 64.3125 
Q 13.671875 62.703125 7.8125 59.421875 
L 7.8125 69.390625 
Q 13.765625 71.78125 18.9375 73 
Q 24.125 74.21875 28.421875 74.21875 
Q 39.75 74.21875 46.484375 68.546875 
Q 53.21875 62.890625 53.21875 53.421875 
Q 53.21875 48.921875 51.53125 44.890625 
Q 49.859375 40.875 45.40625 35.40625 
Q 44.1875 33.984375 37.640625 27.21875 
Q 31.109375 20.453125 19.1875 8.296875 
" id="DejaVuSans-32"/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 1.4 -->
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 1.6 -->
 
 <path d="M 33.015625 40.375 
Q 26.375 40.375 22.484375 35.828125 
Q 18.609375 31.296875 18.609375 23.390625 
Q 18.609375 15.53125 22.484375 10.953125 
Q 26.375 6.390625 33.015625 6.390625 
Q 39.65625 6.390625 43.53125 10.953125 
Q 47.40625 15.53125 47.40625 23.390625 
Q 47.40625 31.296875 43.53125 35.828125 
Q 39.65625 40.375 33.015625 40.375 
M 52.59375 71.296875 
L 52.59375 62.3125 
Q 48.875 64.0625 45.09375 64.984375 
Q 41.3125 65.921875 37.59375 65.921875 
Q 27.828125 65.921875 22.671875 59.328125 
Q 17.53125 52.734375 16.796875 39.40625 
Q 19.671875 43.65625 24.015625 45.921875 
Q 28.375 48.1875 33.59375 48.1875 
Q 44.578125 48.1875 50.953125 41.515625 
Q 57.328125 34.859375 57.328125 23.390625 
Q 57.328125 12.15625 50.6875 5.359375 
Q 44.046875 -1.421875 33.015625 -1.421875 
Q 20.359375 -1.421875 13.671875 8.265625 
Q 6.984375 17.96875 6.984375 36.375 
Q 6.984375 53.65625 15.1875 63.9375 
Q 23.390625 74.21875 37.203125 74.21875 
Q 40.921875 74.21875 44.703125 73.484375 
Q 48.484375 72.75 52.59375 71.296875 
" id="DejaVuSans-36"/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 1.8 -->
 
 <path d="M 31.78125 34.625 
Q 24.75 34.625 20.71875 30.859375 
Q 16.703125 27.09375 16.703125 20.515625 
Q 16.703125 13.921875 20.71875 10.15625 
Q 24.75 6.390625 31.78125 6.390625 
Q 38.8125 6.390625 42.859375 10.171875 
Q 46.921875 13.96875 46.921875 20.515625 
Q 46.921875 27.09375 42.890625 30.859375 
Q 38.875 34.625 31.78125 34.625 
M 21.921875 38.8125 
Q 15.578125 40.375 12.03125 44.71875 
Q 8.5 49.078125 8.5 55.328125 
Q 8.5 64.0625 14.71875 69.140625 
Q 20.953125 74.21875 31.78125 74.21875 
Q 42.671875 74.21875 48.875 69.140625 
Q 55.078125 64.0625 55.078125 55.328125 
Q 55.078125 49.078125 51.53125 44.71875 
Q 48 40.375 41.703125 38.8125 
Q 48.828125 37.15625 52.796875 32.3125 
Q 56.78125 27.484375 56.78125 20.515625 
Q 56.78125 9.90625 50.3125 4.234375 
Q 43.84375 -1.421875 31.78125 -1.421875 
Q 19.734375 -1.421875 13.25 4.234375 
Q 6.78125 9.90625 6.78125 20.515625 
Q 6.78125 27.484375 10.78125 32.3125 
Q 14.79687

---
To use these scenarios in SDDP, we must use a discrete distribution for each timestep. The following function generates `n_scenarios` and returns a vector of `NoiseLaw` corresponding to the evolution of aleas distribution along the time: 

In [9]:
"""Build probability distribution at each timestep based on N scenarios.

Return a Vector{NoiseLaw}"""
function generate_probability_laws(N_STAGES, N_SCENARIOS)
    aleas = zeros(N_SCENARIOS, N_STAGES, 1)
    aleas[:, :, 1] = build_scenarios(N_SCENARIOS, N_STAGES)

    laws = Vector{NoiseLaw}(N_STAGES)

    # uniform probabilities:
    proba = 1/N_SCENARIOS*ones(N_SCENARIOS)

    for t=1:N_STAGES
        aleas_t = reshape(aleas[:, t, :], N_SCENARIOS, 1)'
        laws[t] = NoiseLaw(aleas_t, proba)
    end

    return laws
end

generate_probability_laws (generic function with 1 method)

---

## Solving the problem with SDDP

### SDDP model

We generate 10 scenarios and fit a probability distribution at each timestep: 

In [10]:
N_SCENARIO = 10
aleas = generate_probability_laws(TF-1, 10);

In [11]:
aleas[1].supportSize

10

---
We define the bounds over the state and the control:

In [12]:
x_bounds = [(VOLUME_MIN, VOLUME_MAX), (VOLUME_MIN, VOLUME_MAX)];
u_bounds = [(CONTROL_MIN, CONTROL_MAX), (VOLUME_MIN, VOLUME_MAX)];

and the initial position $X_0$:

In [13]:
x0 = [15, 15]

2-element Array{Int64,1}:
 15
 15

We build an instance of `LinearDynamicLinearCostSPModel` to translate our problem in SDDP: 

In [14]:
model = LinearDynamicLinearCostSPmodel(TF, # number of timestep
                                        u_bounds, # control bounds
                                        x0, # initial state
                                        cost_t, # cost function
                                        dynamic, # dynamic function 
                                        aleas);
set_state_bounds(model, x_bounds);


---
### SDDP parameters

We define the parameters of the algorithm: 

In [15]:
# LP solver: 
solver = ClpSolver()
# Precision of the algorithm in percent: 
EPSILON = 0.05
# Maximum iterations: 
MAX_ITER = 20


params = SDDPparameters(solver, N_SCENARIO, EPSILON, MAX_ITER);

---
### SDDP solving

We could launch the SDDP algorithm to our problem: 

In [16]:
V, pbs = solve_SDDP(model, params, true);

Initialize cuts
Pass number 1	Lower-bound: -3201.4352	Time: 0.11s
Pass number 2	Lower-bound: -3201.4352	Time: 0.05s
Pass number 3	Lower-bound: -3201.4352	Time: 0.05s
Pass number 4	Lower-bound: -3201.4352	Time: 0.05s
Pass number 5	Lower-bound: -3201.4352	Time: 0.05s
Pass number 6	Lower-bound: -3201.4352	Time: 0.06s
Pass number 7	Lower-bound: -3201.4352	Time: 0.05s
Pass number 8	Lower-bound: -3201.4352	Time: 0.06s
Pass number 9	Lower-bound: -3201.4352	Time: 0.06s
Pass number 10	Lower-bound: -3201.4352	Time: 0.06s
Pass number 11	Lower-bound: -3201.4352	Time: 0.06s
Pass number 12	Lower-bound: -3201.4352	Time: 0.06s
Pass number 13	Lower-bound: -3201.4352	Time: 0.06s
Pass number 14	Lower-bound: -3201.4352	Time: 0.07s
Pass number 15	Lower-bound: -3201.4352	Time: 0.07s
Pass number 16	Lower-bound: -3201.4352	Time: 0.08s
Pass number 17	Lower-bound: -3201.4352	Time: 0.07s
Pass number 18	Lower-bound: -3201.4352	Time: 0.07s
Pass number 19	Lower-bound: -3201.4352	Time: 0.06s
Pass number 20	Lower-bou

The algorithm returns the bellman functions (`V`) and a vector of `JuMP.Model` used to approximate these functions with linear cuts. 



---
## Test SDDP with an example 

### Input scenario

We suppose given a scenario of inflows: 


In [17]:
alea_year = Array([7.0 7.0])


1x2 Array{Float64,2}:
 7.0  7.0

We store this scenario as a 3D array, so it could be used to compute a forward-pass:

In [18]:
aleas_scen = zeros(2, 1, 1)
aleas_scen[:, 1, 1] = alea_year;


---
### SDDP simulation

We have only one scenario, so we set the forwardPassNumber equal to 1: 

In [19]:
params.forwardPassNumber = 1

1

Find the optimal control with a forward simulation: 

In [20]:
costs, stocks = forward_simulations(model, params, V, pbs, aleas_scen)

([-3201.435247836369],
3x1x2 Array{Float64,3}:
[:, :, 1] =
 15.0
 27.0
  0.0

[:, :, 2] =
 15.0
 10.0
 30.0,

2x1x2 Array{Float64,3}:
[:, :, 1] =
   5.0
 -20.0

[:, :, 2] =
  0.0
 14.0)

---
### Results 

The cost is: 

In [21]:
SDDP_COST = costs[1]
println("SDDP cost: ", SDDP_COST)

SDDP cost: -3201.435247836369


And the optimal solution is: 

In [23]:
plot(1:TF, stocks[:, :, 1])
plot!(1:TF, stocks[:, :, 2])

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Created with matplotlib (http://matplotlib.org/) -->
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 1.0 -->
 
 
 <path d="M 31.78125 66.40625 
Q 24.171875 66.40625 20.328125 58.90625 
Q 16.5 51.421875 16.5 36.375 
Q 16.5 21.390625 20.328125 13.890625 
Q 24.171875 6.390625 31.78125 6.390625 
Q 39.453125 6.390625 43.28125 13.890625 
Q 47.125 21.390625 47.125 36.375 
Q 47.125 51.421875 43.28125 58.90625 
Q 39.453125 66.40625 31.78125 66.40625 
M 31.78125 74.21875 
Q 44.046875 74.21875 50.515625 64.515625 
Q 56.984375 54.828125 56.984375 36.375 
Q 56.984375 17.96875 50.515625 8.265625 
Q 44.046875 -1.421875 31.78125 -1.421875 
Q 19.53125 -1.421875 13.0625 8.265625 
Q 6.59375 17.96875 6.59375 36.375 
Q 6.59375 54.828125 13.0625 64.515625 
Q 19.53125 74.21875 31.78125 74.21875 
" id="DejaVuSans-30"/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 1.5 -->
 
 <path d="M 10.796875 72.90625 
L 49.515625 72.90625 
L 49.515625 64.59375 
L 19.828125 64.59375 
L 19.828125 46.734375 
Q 21.96875 47.46875 24.109375 47.828125 
Q 26.265625 48.1875 28.421875 48.1875 
Q 40.625 48.1875 47.75 41.5 
Q 54.890625 34.8125 54.890625 23.390625 
Q 54.890625 11.625 47.5625 5.09375 
Q 40.234375 -1.421875 26.90625 -1.421875 
Q 22.3125 -1.421875 17.546875 -0.640625 
Q 12.796875 0.140625 7.71875 1.703125 
L 7.71875 11.625 
Q 12.109375 9.234375 16.796875 8.0625 
Q 21.484375 6.890625 26.703125 6.890625 
Q 35.15625 6.890625 40.078125 11.328125 
Q 45.015625 15.765625 45.015625 23.390625 
Q 45.015625 31 40.078125 35.4375 
Q 35.15625 39.890625 26.703125 39.890625 
Q 22.75 39.890625 18.8125 39.015625 
Q 14.890625 38.140625 10.796875 36.28125 
z
" id="DejaVuSans-35"/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 2.0 -->
 
 <path d="M 19.1875 8.296875 
L 53.609375 8.296875 
L 53.609375 0 
L 7.328125 0 
L 7.328125 8.296875 
Q 12.9375 14.109375 22.625 23.890625 
Q 32.328125 33.6875 34.8125 36.53125 
Q 39.546875 41.84375 41.421875 45.53125 
Q 43.3125 49.21875 43.3125 52.78125 
Q 43.3125 58.59375 39.234375 62.25 
Q 35.15625 65.921875 28.609375 65.921875 
Q 23.96875 65.921875 18.8125 64.3125 
Q 13.671875 62.703125 7.8125 59.421875 
L 7.8125 69.390625 
Q 13.765625 71.78125 18.9375 73 
Q 24.125 74.21875 28.421875 74.21875 
Q 39.75 74.21875 46.484375 68.546875 
Q 53.21875 62.890625 53.21875 53.421875 
Q 53.21875 48.921875 51.53125 44.890625 
Q 49.859375 40.875 45.40625 35.40625 
Q 44.1875 33.984375 37.640625 27.21875 
Q 31.109375 20.453125 19.1875 8.296875 
" id="DejaVuSans-32"/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 2.5 -->
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 3.0 -->
 
 <path d="M 40.578125 39.3125 
Q 47.65625 37.796875 51.625 33 
Q 55.609375 28.21875 55.609375 21.1875 
Q 55.609375 10.40625 48.1875 4.484375 
Q 40.765625 -1.421875 27.09375 -1.421875 
Q 22.515625 -1.421875 17.65625 -0.515625 
Q 12.796875 0.390625 7.625 2.203125 
L 7.625 11.71875 
Q 11.71875 9.328125 16.59375 8.109375 
Q 21.484375 6.890625 26.8125 6.890625 
Q 36.078125 6.890625 40.9375 10.546875 
Q 45.796875 14.203125 45.796875 21.1875 
Q 45.796875 27.640625 41.28125 31.265625 
Q 36.765625 34.90625 28.71875 34.90625 
L 20.21875 34.90625 
L 20.21875 43.015625 
L 29.109375 43.015625 
Q 36.375 43.015625 40.234375 45.921875 
Q 44.09375 48.828125 44.09375 54.296875 
Q 44.09375 59.90625 40.109375 62.90625 
Q 36.140625 65.921875 28.71875 65.921875 
Q 24.65625 65.921875 20.015625 65.03125 
Q 15.375 64.15625 9.8125 62.3125 
L 9.8125 71.09375 
Q 15.4375 72.65625 20.34375 73.4375 
Q 25.25 74.21875 29.59375 74.21875 
Q 40.828125 74.21875 47.359375 69.109375 
Q 53.90625 64.015625 53.90625 55.328125 
Q 53.90625 49.265625 50.4375 45.09375 
Q 46.96875 40.921875 40.578125 39.3125 
" id="DejaVuSans-33"/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 0 -->
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

---
## Comparison with the deterministic solution 
To check the results given by SDDP, we solve the deterministic problem with JuMP: 

In [24]:
m = Model(solver=solver)


@defVar(m,  VOLUME_MIN  <= x1[1:(TF)]  <= VOLUME_MAX)
@defVar(m,  VOLUME_MIN  <= x2[1:(TF)]  <= VOLUME_MAX)
@defVar(m,  CONTROL_MIN <= u1[1:TF-1]  <= CONTROL_MAX)
@defVar(m, VOLUME_MIN  <= u2[1:TF-1] <= VOLUME_MAX)

@setObjective(m, Min, sum{COST[i]*(u1[i]), i = 1:TF-1})

for i in 1:TF-1
    @addConstraint(m, x1[i+1] - x1[i] - u1[i] + u2[i] - alea_year[i] == 0)
    @addConstraint(m, x2[i+1] - x2[i] + u1[i] == 0)
end

@addConstraint(m, x1[1] == x0[1])
@addConstraint(m, x2[1] == x0[2])

status = solve(m)
println(status)

LP_COST = getObjectiveValue(m)
println("LP value: ", LP_COST)

Optimal
LP value: -3201.43524783637


And we plot the evolution of the stocks:

In [25]:
plot(getValue(x1))
plot!(getValue(x2))

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Created with matplotlib (http://matplotlib.org/) -->
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 1.0 -->
 
 
 <path d="M 31.78125 66.40625 
Q 24.171875 66.40625 20.328125 58.90625 
Q 16.5 51.421875 16.5 36.375 
Q 16.5 21.390625 20.328125 13.890625 
Q 24.171875 6.390625 31.78125 6.390625 
Q 39.453125 6.390625 43.28125 13.890625 
Q 47.125 21.390625 47.125 36.375 
Q 47.125 51.421875 43.28125 58.90625 
Q 39.453125 66.40625 31.78125 66.40625 
M 31.78125 74.21875 
Q 44.046875 74.21875 50.515625 64.515625 
Q 56.984375 54.828125 56.984375 36.375 
Q 56.984375 17.96875 50.515625 8.265625 
Q 44.046875 -1.421875 31.78125 -1.421875 
Q 19.53125 -1.421875 13.0625 8.265625 
Q 6.59375 17.96875 6.59375 36.375 
Q 6.59375 54.828125 13.0625 64.515625 
Q 19.53125 74.21875 31.78125 74.21875 
" id="DejaVuSans-30"/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 1.5 -->
 
 <path d="M 10.796875 72.90625 
L 49.515625 72.90625 
L 49.515625 64.59375 
L 19.828125 64.59375 
L 19.828125 46.734375 
Q 21.96875 47.46875 24.109375 47.828125 
Q 26.265625 48.1875 28.421875 48.1875 
Q 40.625 48.1875 47.75 41.5 
Q 54.890625 34.8125 54.890625 23.390625 
Q 54.890625 11.625 47.5625 5.09375 
Q 40.234375 -1.421875 26.90625 -1.421875 
Q 22.3125 -1.421875 17.546875 -0.640625 
Q 12.796875 0.140625 7.71875 1.703125 
L 7.71875 11.625 
Q 12.109375 9.234375 16.796875 8.0625 
Q 21.484375 6.890625 26.703125 6.890625 
Q 35.15625 6.890625 40.078125 11.328125 
Q 45.015625 15.765625 45.015625 23.390625 
Q 45.015625 31 40.078125 35.4375 
Q 35.15625 39.890625 26.703125 39.890625 
Q 22.75 39.890625 18.8125 39.015625 
Q 14.890625 38.140625 10.796875 36.28125 
z
" id="DejaVuSans-35"/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 2.0 -->
 
 <path d="M 19.1875 8.296875 
L 53.609375 8.296875 
L 53.609375 0 
L 7.328125 0 
L 7.328125 8.296875 
Q 12.9375 14.109375 22.625 23.890625 
Q 32.328125 33.6875 34.8125 36.53125 
Q 39.546875 41.84375 41.421875 45.53125 
Q 43.3125 49.21875 43.3125 52.78125 
Q 43.3125 58.59375 39.234375 62.25 
Q 35.15625 65.921875 28.609375 65.921875 
Q 23.96875 65.921875 18.8125 64.3125 
Q 13.671875 62.703125 7.8125 59.421875 
L 7.8125 69.390625 
Q 13.765625 71.78125 18.9375 73 
Q 24.125 74.21875 28.421875 74.21875 
Q 39.75 74.21875 46.484375 68.546875 
Q 53.21875 62.890625 53.21875 53.421875 
Q 53.21875 48.921875 51.53125 44.890625 
Q 49.859375 40.875 45.40625 35.40625 
Q 44.1875 33.984375 37.640625 27.21875 
Q 31.109375 20.453125 19.1875 8.296875 
" id="DejaVuSans-32"/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 2.5 -->
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 3.0 -->
 
 <path d="M 40.578125 39.3125 
Q 47.65625 37.796875 51.625 33 
Q 55.609375 28.21875 55.609375 21.1875 
Q 55.609375 10.40625 48.1875 4.484375 
Q 40.765625 -1.421875 27.09375 -1.421875 
Q 22.515625 -1.421875 17.65625 -0.515625 
Q 12.796875 0.390625 7.625 2.203125 
L 7.625 11.71875 
Q 11.71875 9.328125 16.59375 8.109375 
Q 21.484375 6.890625 26.8125 6.890625 
Q 36.078125 6.890625 40.9375 10.546875 
Q 45.796875 14.203125 45.796875 21.1875 
Q 45.796875 27.640625 41.28125 31.265625 
Q 36.765625 34.90625 28.71875 34.90625 
L 20.21875 34.90625 
L 20.21875 43.015625 
L 29.109375 43.015625 
Q 36.375 43.015625 40.234375 45.921875 
Q 44.09375 48.828125 44.09375 54.296875 
Q 44.09375 59.90625 40.109375 62.90625 
Q 36.140625 65.921875 28.71875 65.921875 
Q 24.65625 65.921875 20.015625 65.03125 
Q 15.375 64.15625 9.8125 62.3125 
L 9.8125 71.09375 
Q 15.4375 72.65625 20.34375 73.4375 
Q 25.25 74.21875 29.59375 74.21875 
Q 40.828125 74.21875 47.359375 69.109375 
Q 53.90625 64.015625 53.90625 55.328125 
Q 53.90625 49.265625 50.4375 45.09375 
Q 46.96875 40.921875 40.578125 39.3125 
" id="DejaVuSans-33"/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 0 -->
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

The solution given by the solver is more optimistic, as it assumes that the future is known in advance. 

If we consider the costs, we have a discrepancy between the two solutions :


In [26]:
abs((LP_COST - SDDP_COST)/LP_COST)


2.840896758376085e-16

## Comparison with SDP 
To check the results given by SDDP, we solve the stochastic problem with SDP: 

We define constants necessary for SDP

In [27]:
N_CONTROLS = 2;
N_STATES = 2;
N_NOISES = 1;

We have to define a constraint function as SDP may take into account mixed state and control constraints

In [28]:
function constraints(t, x, u, w)
    return (x[1]<=VOLUME_MAX)&(x[1]>=VOLUME_MIN)&(x[2]<=VOLUME_MAX)&(x[2]>=VOLUME_MIN)
end


constraints (generic function with 1 method)

We define a final cost function as well

In [29]:
function finalCostFunction(x)
    return 0.
end

finalCostFunction (generic function with 1 method)

In [30]:
modelSDP = StochDynProgModel(TF,
                    x_bounds, u_bounds,
                    x0, cost_t,
                    finalCostFunction, dynamic,
                    constraints, aleas);

We define some parameters needed by SDP : states and controls discretization and monte carlo sampling size

In [31]:
stateSteps = [1,1];
controlSteps = [1,1];
monteCarloSize = 10;

We have to define wether the controls are chosen knowing the uncertainty realization (hazard-decision) or prior 
to know the uncertainty realization (decision-hazard). We chose here hazard decision to compare with SDDP.

In [50]:
infoStruct = "HD";

Let's define the parameters of the algorithm 

In [51]:
paramsSDP = SDPparameters(modelSDP, stateSteps, controlSteps,  infoStruct, "Exact", monteCarloSize);

We are ready to produce the value functions

In [52]:
V_sdp = solve_DP(modelSDP, paramsSDP, true);

Starting stochastic dynamic programming decision hazard computation
Progress: 100% Time: 0:00:59


Thanks to these value functions we are now able to compute a forward simulation with the same scenario

In [53]:
costs_sdp, stocks_sdp, controls_sdp = sdp_forward_simulation(modelSDP, paramsSDP, aleas_scen, V_sdp, true )

([-3201.43524783637],
3x1x2 Array{Float64,3}:
[:, :, 1] =
 15.0
 27.0
 14.0

[:, :, 2] =
 15.0
 10.0
 30.0,

2x1x2 Array{Float64,3}:
[:, :, 1] =
   5.0
 -20.0

[:, :, 2] =
 0.0
 0.0)

We can compare this result to the deterministic one and SDDP one

In [54]:
plot(stocks_sdp[:,:,1])
plot!(stocks_sdp[:,:,2])

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Created with matplotlib (http://matplotlib.org/) -->
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 1.0 -->
 
 
 <path d="M 31.78125 66.40625 
Q 24.171875 66.40625 20.328125 58.90625 
Q 16.5 51.421875 16.5 36.375 
Q 16.5 21.390625 20.328125 13.890625 
Q 24.171875 6.390625 31.78125 6.390625 
Q 39.453125 6.390625 43.28125 13.890625 
Q 47.125 21.390625 47.125 36.375 
Q 47.125 51.421875 43.28125 58.90625 
Q 39.453125 66.40625 31.78125 66.40625 
M 31.78125 74.21875 
Q 44.046875 74.21875 50.515625 64.515625 
Q 56.984375 54.828125 56.984375 36.375 
Q 56.984375 17.96875 50.515625 8.265625 
Q 44.046875 -1.421875 31.78125 -1.421875 
Q 19.53125 -1.421875 13.0625 8.265625 
Q 6.59375 17.96875 6.59375 36.375 
Q 6.59375 54.828125 13.0625 64.515625 
Q 19.53125 74.21875 31.78125 74.21875 
" id="DejaVuSans-30"/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 1.5 -->
 
 <path d="M 10.796875 72.90625 
L 49.515625 72.90625 
L 49.515625 64.59375 
L 19.828125 64.59375 
L 19.828125 46.734375 
Q 21.96875 47.46875 24.109375 47.828125 
Q 26.265625 48.1875 28.421875 48.1875 
Q 40.625 48.1875 47.75 41.5 
Q 54.890625 34.8125 54.890625 23.390625 
Q 54.890625 11.625 47.5625 5.09375 
Q 40.234375 -1.421875 26.90625 -1.421875 
Q 22.3125 -1.421875 17.546875 -0.640625 
Q 12.796875 0.140625 7.71875 1.703125 
L 7.71875 11.625 
Q 12.109375 9.234375 16.796875 8.0625 
Q 21.484375 6.890625 26.703125 6.890625 
Q 35.15625 6.890625 40.078125 11.328125 
Q 45.015625 15.765625 45.015625 23.390625 
Q 45.015625 31 40.078125 35.4375 
Q 35.15625 39.890625 26.703125 39.890625 
Q 22.75 39.890625 18.8125 39.015625 
Q 14.890625 38.140625 10.796875 36.28125 
z
" id="DejaVuSans-35"/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 2.0 -->
 
 <path d="M 19.1875 8.296875 
L 53.609375 8.296875 
L 53.609375 0 
L 7.328125 0 
L 7.328125 8.296875 
Q 12.9375 14.109375 22.625 23.890625 
Q 32.328125 33.6875 34.8125 36.53125 
Q 39.546875 41.84375 41.421875 45.53125 
Q 43.3125 49.21875 43.3125 52.78125 
Q 43.3125 58.59375 39.234375 62.25 
Q 35.15625 65.921875 28.609375 65.921875 
Q 23.96875 65.921875 18.8125 64.3125 
Q 13.671875 62.703125 7.8125 59.421875 
L 7.8125 69.390625 
Q 13.765625 71.78125 18.9375 73 
Q 24.125 74.21875 28.421875 74.21875 
Q 39.75 74.21875 46.484375 68.546875 
Q 53.21875 62.890625 53.21875 53.421875 
Q 53.21875 48.921875 51.53125 44.890625 
Q 49.859375 40.875 45.40625 35.40625 
Q 44.1875 33.984375 37.640625 27.21875 
Q 31.109375 20.453125 19.1875 8.296875 
" id="DejaVuSans-32"/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 2.5 -->
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 3.0 -->
 
 <path d="M 40.578125 39.3125 
Q 47.65625 37.796875 51.625 33 
Q 55.609375 28.21875 55.609375 21.1875 
Q 55.609375 10.40625 48.1875 4.484375 
Q 40.765625 -1.421875 27.09375 -1.421875 
Q 22.515625 -1.421875 17.65625 -0.515625 
Q 12.796875 0.390625 7.625 2.203125 
L 7.625 11.71875 
Q 11.71875 9.328125 16.59375 8.109375 
Q 21.484375 6.890625 26.8125 6.890625 
Q 36.078125 6.890625 40.9375 10.546875 
Q 45.796875 14.203125 45.796875 21.1875 
Q 45.796875 27.640625 41.28125 31.265625 
Q 36.765625 34.90625 28.71875 34.90625 
L 20.21875 34.90625 
L 20.21875 43.015625 
L 29.109375 43.015625 
Q 36.375 43.015625 40.234375 45.921875 
Q 44.09375 48.828125 44.09375 54.296875 
Q 44.09375 59.90625 40.109375 62.90625 
Q 36.140625 65.921875 28.71875 65.921875 
Q 24.65625 65.921875 20.015625 65.03125 
Q 15.375 64.15625 9.8125 62.3125 
L 9.8125 71.09375 
Q 15.4375 72.65625 20.34375 73.4375 
Q 25.25 74.21875 29.59375 74.21875 
Q 40.828125 74.21875 47.359375 69.109375 
Q 53.90625 64.015625 53.90625 55.328125 
Q 53.90625 49.265625 50.4375 45.09375 
Q 46.96875 40.921875 40.578125 39.3125 
" id="DejaVuSans-33"/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 10 -->
 
 
 
 
 
 
 
 
 
 
 
 
 
 


In [55]:
abs((costs_sdp - SDDP_COST)/costs_sdp)

1x1 Array{Float64,2}:
 2.8409e-16

In [56]:
abs((costs_sdp - LP_COST)/LP_COST)

1-element Array{Float64,1}:
 0.0

## Comparison with the Extensive Formulation 
We now compare the results with the Extensive Formulation

In [57]:
costs_EF, EFstatus = extensive_formulation(model,params)

(-6759.768877076356,:Optimal)

In [58]:
abs((costs_EF - SDDP_COST)/costs_EF)

0.526398711841608

In [59]:
abs((costs_EF - costs_sdp)/costs_EF)

1-element Array{Float64,1}:
 0.526399